In [1]:
import math

import torch
import torch.nn as nn

from diffusers.optimization import get_scheduler
from tqdm import tqdm

import pandas as pd
import numpy as np

import random

import os
device = "cuda"
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(0)


/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Random seed set as 0


In [2]:
save_location = "/media/bhux/alpha/xsd_mvp/test/"

# uniform, lg, gaussian

NUMERICAL = 12
CATEGORICAL = 0

INFILLING_TYPE = ''
NOISE_TYPE = ''

In [3]:


def m(v):
    nonnull = v[np.isnan(v) == False]
    return np.mean(nonnull)

def std(v):
    nonnull = v[np.isnan(v) == False]
    return np.std(nonnull)

def infill_null(v):
    v[np.isnan(v)] = 0
    return v

def remove_outliers(lists):
    b = np.ones(lists[0].shape)

    for l in lists:
        q1 = np.nanquantile(l,0.25)
        q3 = np.nanquantile(l,0.75)

        iqr = q3 - q1
        lower = q1 - 1.5*iqr
        upper = q3 + 1.5*iqr

        b = np.logical_and(b, np.logical_or(l > lower , np.isnan(l)))
        b = np.logical_and(b, np.logical_or(l < upper , np.isnan(l)))
    
    return b

def norm(v):
    nonnull = v[np.isnan(v) == False]
    max = np.nanmax(nonnull)
    min = np.nanmin(nonnull)

    return 2*((v - min) / (max - min))-1

def get_local_gaussian(ys, numbins=50):
    max = np.nanmax(ys)
    min = np.nanmin(ys)
    bins = np.linspace(min, max, retstep=numbins)[0]
    s_ys = np.array(sorted(ys, reverse=True))

    d, m, s = [], [], []
    for i in range(numbins-1):
        low = bins[i]
        high = bins[i+1]
        tbool = np.logical_and(low<=s_ys, s_ys<=high)
        data = s_ys[tbool]
        d.append(len(data))
        m.append(data.mean() if not np.isnan(data.mean()) else low)
        s.append(data.std() if not np.isnan(data.std()) else 0.01)
    d = np.array(d) / sum(d)

    return d, np.array(m), np.array(s)

def convert_categorical(ys, numbins=50):
    max = np.nanmax(ys)
    min = np.nanmin(ys)
    bins = np.linspace(min, max, retstep=numbins)[0]
    s = np.array(sorted(enumerate(ys), key=lambda x:x[1]))
    s_inds = s[:,0].astype(int)
    s_ys = s[:,1]
    
    
    nys = ys
    for i in range(numbins-1):
        low = bins[i]
        high = bins[i+1]
        tbool = np.logical_and(low<=s_ys, s_ys<=high)
        
        if sum(tbool) > 1:
            nys[np.array(s_inds[tbool])] = i+1

    print(i+1)
    nys[np.isnan(nys)] = 0
    return nys

def sample_local_gaussian(v, numbins=50):
    d,m,s = get_local_gaussian(v, numbins=numbins)
    num = sum(np.isnan(v))

    samples = np.random.choice(numbins-1, num, p=d)
    rand_n = np.random.randn(num)

    adjust = m[samples] + 1.2 * rand_n *s[samples]
    
    # override 
    #adjust = np.zeros(adjust.shape)
    
    if INFILLING_TYPE == "zeros":
        adjust = np.zeros(adjust.shape)

    if INFILLING_TYPE == "uniform":
        adjust = np.random.uniform(low=-1.0, high=1.0, size=adjust.shape)

    if INFILLING_TYPE == "gaussian":
        adjust = np.random.normal(size=adjust.shape)

    v[np.isnan(v)] = adjust
    return v, (d,m,s)


def sample_noise(b, dmss, numbins=50):
    if NOISE_TYPE == "uniform":
        return np.random.uniform(low=-1.0, high=1.0, size=(b,NUMERICAL))
    
    if NOISE_TYPE == "gaussian":
        return np.random.normal(size=(b,NUMERICAL))

    vs = []
    for d,m,s in dmss:
        samples = np.random.choice(numbins-1, b, p=d)
        rand_n = np.random.randn(b)
        vs.append(m[samples] + 1.2 * rand_n *s[samples])
    return np.stack(vs, axis=-1)

def unison_shuffled_copies(a, b):
    #assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return np.array(a)[p], np.array(b)[p]

In [4]:
PATH = "./data/xtended_data_all.csv"
EMB_PATH = "./data/xtended_emb_all_deberta_pubchem.npy"

f = pd.read_csv(PATH)
drug_embeddings = np.load(EMB_PATH)
smiles = f['Drug'].values
vlists = {
    col: f[col].values for col in f.drop(labels=['Drug'], axis=1).columns[:NUMERICAL] 
}

inmask = remove_outliers([v for _,v in vlists.items()])
print(sum(inmask))
smiles = smiles[inmask]
vlists = {
    k: v[inmask] for k,v in vlists.items()
}

vlists = {
    k: norm(v) for k,v in vlists.items()
}

# for col in f.drop(labels=['Drug'], axis=1).columns[:NUMERICAL]:
#     vlists[col+"_cat"] = vlists[col]

nullmask = np.stack([
    np.isnan(v)==False for _,v in vlists.items()
    ], axis=-1)

dmss = []
for k,v in vlists.items():
    vlists[k], dms = sample_local_gaussian(v, numbins=15)
    dmss.append(dms)

for col in f.drop(labels=['Drug'], axis=1).columns[:NUMERICAL]:
    vlists[col+"_cat"] = convert_categorical(vlists[col], numbins=15)

# for col in f.drop(labels=['Drug'], axis=1).columns[NUMERICAL:]:
#     nan = np.isnan(vlists[col])
#     vlists[col] += 1
#     vlists[col][nan] = 0

# dmss = []
# for k,v in vlists.items():
#     dms = get_local_gaussian(v, numbins=50)
#     dmss.append(dms)

dataset = []
for i, gt in enumerate(zip(*[v for _,v in vlists.items()])):
    dataset.append({
        "sm": smiles[i],
        "ft": drug_embeddings[i],
        "ma": nullmask[i],
        "gt": np.array(gt),
        "od": np.array(gt[NUMERICAL:]),
    })
    # print(gt)
    # print(nullmask[i])
    # break

valCount = np.sum(nullmask, axis=0)*0.1
dataset, rcomb = unison_shuffled_copies(dataset, nullmask)
trdataset = []
valdataset = []
for c, d in zip(rcomb, dataset):
    inc = False
    for i, j in enumerate(list(c)):
        if j and valCount[i] > 0:
            valCount[i] -= 1
            inc = True
    if inc:
        valdataset.append(d)
    else:
        trdataset.append(d)

print(len(trdataset))
print(len(valdataset))
print(len(list(vlists.keys())))

28422
14
14


/tmp/ipykernel_256404/1209130151.py:49: RuntimeWarning: Mean of empty slice.
  m.append(data.mean() if not np.isnan(data.mean()) else low)
/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/numpy/_core/_methods.py:227: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/numpy/_core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/numpy/_core/_methods.py:219: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


14
14
14
14
14
14
14
14
14
14
25531
2891
24


In [5]:
from torch.utils.data import Dataset

class GaucamolDataset(Dataset):
    def __init__(self, dataset) -> None:
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]
    
    def update(self, idx, delta):
        item = self.dataset[idx]["gt"]
        self.dataset[idx]["gt"] = item + delta

trainset = GaucamolDataset(trdataset)
valset = GaucamolDataset(valdataset)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=128, shuffle=True)
steps_per_epoch = len(trainset)
DMSS = dmss[:NUMERICAL]

In [6]:
def train(diffusion, ema, gamma, dataloader, optimizer, lr_scheduler, two_noise=False):
    diffusion.train()
    running_loss = 0
    global_step = 0
    for i, batch in enumerate(dataloader):
        ft = batch['ft'].to(device).float()
        gt = batch['gt'].to(device).float()
        od = batch['od'].to(device).long()
        mask = batch['ma'].to(device)
        bs = ft.shape[0]

        optimizer.zero_grad()
        loss_multi, loss_gauss = diffusion.mixed_loss(ft, gt, od, mask, DMSS)

        loss = loss_multi + loss_gauss
        
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        ema.update_params(gamma)
        gamma = ema.update_gamma(global_step)

        running_loss += loss.item()
        global_step += 1
    return running_loss/global_step

In [7]:
from sklearn.metrics import mean_squared_error
import csv
from utils import ohe_to_categories

def evaluate(e, ema, dataloader):
    ema.ema_model.eval()
    before_mse = 0
    running_mse = 0
    global_step = 0
    vals = {}
    device = 'cuda'
    ema.ema_model.to(device)
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            sm = batch['sm']
            mask = batch['ma'].repeat(1,2)
            ft = batch['ft'].to(device).float()
            gt = batch['gt'].to(device).float()
            od = batch['od'].to(device).long()
            bs = ft.shape[0]

            x_in, generated_ys = ema.ema_model.sample(ft, bs, od, DMSS, clip_sample=True)

            raw_mse = mean_squared_error(gt[mask].flatten().cpu(), x_in[mask].flatten().cpu())
            mse = mean_squared_error(gt[mask].flatten().cpu(), generated_ys[mask].flatten().cpu())

            for s, g in zip(sm, list(generated_ys.cpu().numpy())):
                vals[s] = g
            
            before_mse += raw_mse
            running_mse += mse
            global_step += 1

    with open(save_location+'{}_dict.csv'.format(e), 'w') as csv_file:  
        writer = csv.writer(csv_file)
        for key, value in vals.items():
            writer.writerow([key, value])

    return running_mse / global_step, before_mse / global_step
            

In [8]:
import logging
import sys
import optuna
from sdt import SDT
from diffusion import GaussianMultinomialDiffusion
torch.set_printoptions(profile="full")
from ema import EMA

num_epochs = 10

def objective(trial):
    lr = trial.suggest_float(
        "lr", 1e-5, 1e-2, log=True
    ) #0.0005

    wd = trial.suggest_float(
        "wd", 1e-5, 1e-2, log=True
    ) #1e-4

    warmup = trial.suggest_categorical(
        "warmup", [50, 100, 150, 200, 250, 300]
    ) #200

    gamma = trial.suggest_float(
        "gamma", 0.97, 0.999, log=True
    ) #0.994

    total_num_steps = (steps_per_epoch * num_epochs)

    model = SDT(
        time_dim = trial.suggest_categorical(
            "time_dim", [16, 32, 64]
        ), #64,
        cond_size = 768,
        patch_size = trial.suggest_categorical(
            "patch_size", [8, 16, 32, 64]
        ), #16
        y_dim = NUMERICAL+15*(NUMERICAL+CATEGORICAL),
        dim = 768,
        depth = trial.suggest_int(
            "depth", 4, 12
        ), #8,
        heads = trial.suggest_int(
            "heads", 4, 12
        ), #8,
        mlp_dim = trial.suggest_categorical(
            "mlp_dim", [256, 512, 768, 1024]
        ), #768,
        dropout = 0.1,
        emb_dropout = trial.suggest_float(
            "emb_dropout", 0, 0.2
        ), #0.1,
        num_classes = 15,
    )
    model.to(device)

    total_params = sum(p.numel() for p in model.parameters())
    print(f"Number of parameters: {total_params}")

    diffusion = GaussianMultinomialDiffusion(
        num_classes = np.array([15 for _ in range(NUMERICAL+CATEGORICAL)]),
        num_numerical_features = NUMERICAL,
        denoise_fn = model,
        device = device,
    )
    diffusion.to(device)

    ema = EMA(diffusion, gamma, total_num_steps)

    optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=lr,
            weight_decay=wd,
        )

    lr_scheduler = get_scheduler(
            trial.suggest_categorical(
                "schedule", 
                [
                    "cosine", 
                    "linear", 
                    "cosine_with_restarts", 
                    "constant", 
                    "constant_with_warmup",
                ]
            ), #"cosine",
            optimizer=optimizer,
            num_warmup_steps=warmup,
            num_training_steps=total_num_steps,
        )
    
    l = ""
    loss = 0
    for e in range(num_epochs):
        loss = train(diffusion, ema, gamma, trainloader, optimizer, lr_scheduler)

    mse, bmse = evaluate(e, ema, valloader)
    return mse


In [9]:
import pickle
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "2024-10-25-optimize-ximagand"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

study = optuna.create_study(study_name=study_name, storage=storage_name, direction="minimize", load_if_exists=True)
study.optimize(objective, n_trials=900)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

with open("2024-10-25-sampler.pkl", "wb") as fout:
    pickle.dump(study.sampler, fout)

[I 2024-10-25 15:02:33,251] Using an existing study with name '2024-10-25-optimize-ximagand' instead of creating a new one.


Using an existing study with name '2024-10-25-optimize-ximagand' instead of creating a new one.
Number of parameters: 19334528
torch.Size([180])
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0


[I 2024-10-25 15:04:37,527] Trial 7 finished with value: 23.35610762237664 and parameters: {'lr': 0.0022843538000994427, 'wd': 0.0002708728156692418, 'warmup': 200, 'gamma': 0.9772592840217776, 'time_dim': 16, 'patch_size': 32, 'depth': 6, 'heads': 8, 'mlp_dim': 256, 'emb_dropout': 0.11221196434678829, 'schedule': 'cosine'}. Best is trial 1 with value: 23.05985780360234.


Trial 7 finished with value: 23.35610762237664 and parameters: {'lr': 0.0022843538000994427, 'wd': 0.0002708728156692418, 'warmup': 200, 'gamma': 0.9772592840217776, 'time_dim': 16, 'patch_size': 32, 'depth': 6, 'heads': 8, 'mlp_dim': 256, 'emb_dropout': 0.11221196434678829, 'schedule': 'cosine'}. Best is trial 1 with value: 23.05985780360234.
Number of parameters: 19977536
torch.Size([180])
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0


[I 2024-10-25 15:06:28,114] Trial 8 finished with value: 27.68829737941495 and parameters: {'lr': 0.006556702953383152, 'wd': 0.0007352947882873483, 'warmup': 200, 'gamma': 0.9945809272965487, 'time_dim': 64, 'patch_size': 64, 'depth': 7, 'heads': 4, 'mlp_dim': 512, 'emb_dropout': 0.16841374802502804, 'schedule': 'constant'}. Best is trial 1 with value: 23.05985780360234.


...

[W 2024-10-25 15:13:32,028] Trial 11 failed with parameters: {'lr': 1.4697450557892006e-05, 'wd': 3.480354635968786e-05, 'warmup': 150, 'gamma': 0.9981862245135316, 'time_dim': 32, 'patch_size': 32, 'depth': 4, 'heads': 12, 'mlp_dim': 1024, 'emb_dropout': 0.008672196331743487, 'schedule': 'reduce_lr_on_plateau'} because of the following error: ValueError("'reduce_lr_on_plateau' is not a valid SchedulerType").
Traceback (most recent call last):
  File "/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_256404/2343495787.py", line 76, in objective
    lr_scheduler = get_scheduler(
                   ^^^^^^^^^^^^^^
  File "/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/diffusers/optimization.py", line 322, in get_scheduler
    name = SchedulerType(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/bhux/anaconda3/envs/s2pk/lib/

Number of parameters: 20959232
torch.Size([180])
Trial 11 failed with parameters: {'lr': 1.4697450557892006e-05, 'wd': 3.480354635968786e-05, 'warmup': 150, 'gamma': 0.9981862245135316, 'time_dim': 32, 'patch_size': 32, 'depth': 4, 'heads': 12, 'mlp_dim': 1024, 'emb_dropout': 0.008672196331743487, 'schedule': 'reduce_lr_on_plateau'} because of the following error: ValueError("'reduce_lr_on_plateau' is not a valid SchedulerType").
Traceback (most recent call last):
  File "/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_256404/2343495787.py", line 76, in objective
    lr_scheduler = get_scheduler(
                   ^^^^^^^^^^^^^^
  File "/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/diffusers/optimization.py", line 322, in get_scheduler
    name = SchedulerType(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/bhux/ana

[W 2024-10-25 15:13:32,029] Trial 11 failed with value None.


Trial 11 failed with value None.


ValueError: 'reduce_lr_on_plateau' is not a valid SchedulerType